In [1]:
"""import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        """

"import tensorflow as tf\ngpus = tf.config.list_physical_devices('GPU')\nif gpus:\n    try:\n        for gpu in gpus:\n            tf.config.experimental.set_memory_growth(gpu, True)\n    except RuntimeError as e:\n        print(e)\n        "

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])  # 4GB limit
    except RuntimeError as e:
        print(e)


2025-06-23 18:59:35.445263: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750705175.481123  317413 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750705175.491987  317413 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750705175.531512  317413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750705175.531546  317413 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750705175.531548  317413 computation_placer.cc:177] computation placer alr

In [3]:
NUM_SENTENCES = 42 # Use only the first 20,000 records.
MAX_NUM_WORDS = 500 # Use 20,000 words for tokenizing
MAX_SENT_LEN = 150

EMBEDDING_SIZE = 100

LSTM_NEURONS = 512

latent_dims = [LSTM_NEURONS, 256, 128, 64, 32]

BATCH_SIZE = 32
EPOCHS = 500



In [4]:
import csv
def read_text_file(filename): 
    lines = []
    try:
        with open(filename, newline='\n', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            for row in csv_reader:
              lines.append(row)
        #print(len(lines))
        #print(lines[0])
        return lines
    except FileNotFoundError:
        return f"Error: The file '{filename}' was not found."
    except Exception as e:
        return f"An error occurred: {e}"
    

file_name = '/mnt/d/Work/wsl/ML-DS/english_to_bangla.csv'
# Read the content of the file
lines = read_text_file(file_name)
 

In [5]:
lines[1]

['a child in a pink dress is climbing up a set of stairs in an entry way .',
 'একটি গোলাপী জামা পরা বাচ্চা মেয়ে একটি বাড়ির প্রবেশ পথের সিঁড়ি বেয়ে উঠছে।']

In [6]:
inputs = []
outputs_i = []
outputs = []

 
count = 0
for line in lines: 
    if count > NUM_SENTENCES:
        break
    count += 1 
    ip = line[0].rstrip()
    temp_op = line[1].rstrip()
    op_i = '<sos> '+temp_op
    op = temp_op+' <eos>'
    inputs.append(ip)
    outputs_i.append(op_i)
    outputs.append(op)

print('Total inputs =', len(inputs))
print('Total intermediate outputs =', len(outputs_i))
print('Total outputs =', len(outputs))
print('\nSample:')
print(inputs[10])
print(outputs_i[10])
print(outputs[10])

Total inputs = 43
Total intermediate outputs = 43
Total outputs = 43

Sample:
two dogs on pavement moving toward each other .
<sos> রাস্তার পাশে দুইটি কুকুর পরস্পরের দিকে এগিয়ে যাচ্ছ।
রাস্তার পাশে দুইটি কুকুর পরস্পরের দিকে এগিয়ে যাচ্ছ। <eos>


In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer

input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(inputs)

inputs_seq = input_tokenizer.texts_to_sequences(inputs)

inputs_word2index = input_tokenizer.word_index
print('Total unique words in input:', len(inputs_word2index))

inputs_numwords = len(inputs_word2index)+1

inputs_maxlen = max(len(s) for s in inputs_seq)
print('Length of longest sentence in input:', inputs_maxlen)

output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

outputs_i_seq = output_tokenizer.texts_to_sequences(outputs_i)
outputs_seq = output_tokenizer.texts_to_sequences(outputs)

outputs_word2index = output_tokenizer.word_index
print('Total unique words in output:', len(outputs_word2index))

outputs_numwords = len(outputs_word2index)+1

outputs_maxlen = max(len(s) for s in outputs_seq)
print('Length of longest sentence in output:', outputs_maxlen)

Total unique words in input: 154
Length of longest sentence in input: 20
Total unique words in output: 213
Length of longest sentence in output: 20


In [8]:
from keras.preprocessing.sequence import pad_sequences

encoder_input_sequences = pad_sequences(inputs_seq, maxlen=inputs_maxlen)
print('encoder_input_sequences shape:', encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(outputs_i_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_inputs_sequences shape:', decoder_input_sequences.shape)

decoder_output_sequences = pad_sequences(outputs_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_output_sequences shape:', decoder_output_sequences.shape)



encoder_input_sequences shape: (43, 20)
decoder_inputs_sequences shape: (43, 20)
decoder_output_sequences shape: (43, 20)


In [9]:
from numpy import asarray, zeros
 
embeddings_dict = dict()
glove_file = open(f'/mnt/d/Work/wsl/ML-DS/glove.6B.{EMBEDDING_SIZE}d.txt', encoding='utf-8') 

for line in glove_file:
    records = line.split()
    word = records[0]
    vector = asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector

glove_file.close()

num_words = min(MAX_NUM_WORDS, len(inputs_word2index)+1)

embedding_matrix = zeros((num_words, EMBEDDING_SIZE))

for word, index in inputs_word2index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [10]:
from keras.layers import Input, Embedding, LSTM, Dense

model_mode = 2


In [11]:
## Muti layer LSTM
from tensorflow.keras import layers, regularizers
dropout_rate = 0.5
l2_lambda = 0.01

if (model_mode == 2) :
    encoder_embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=inputs_maxlen)
    decoder_embedding_layer = Embedding(outputs_numwords, LSTM_NEURONS)
    
    decoder_outputs_onehot = zeros((len(inputs), outputs_maxlen, outputs_numwords), dtype='float32')
    print('decoder_outputs_onehot shape:', decoder_outputs_onehot.shape)
    
    for i, d in enumerate(decoder_output_sequences):
        for t, w in enumerate(d):
            decoder_outputs_onehot[i, t, w] = 1
    
    encoder_inputs = Input(shape=(inputs_maxlen,))
    encoder_inputs_emb = encoder_embedding_layer(encoder_inputs)

    encoder_states = []
    x = encoder_inputs_emb
    for i in range(len(latent_dims)):
        return_seq = (i < len(latent_dims) - 1)
        x = layers.BatchNormalization()(x) 
        x, h, c = LSTM(latent_dims[i], return_state=True, kernel_regularizer=regularizers.l2(l2_lambda), dropout=dropout_rate, return_sequences=return_seq)(x)
        encoder_states += [h, c]
        
    # First LSTM layer (returns sequences for stacking)
    #encoder_lstm1 = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
    #encoder_outputs1, h1, c1 = encoder_lstm1(encoder_inputs_emb)
    
    # Second LSTM layer (returns last state only)
    #encoder_lstm2 = LSTM(LSTM_NEURONS, return_state=True)
    #encoder_outputs2, h2, c2 = encoder_lstm2(encoder_outputs1)
    #encoder_states = [h2, c2]  # Use the states from the last LSTM layer

    # Decoder
    decoder_inputs = Input(shape=(outputs_maxlen,))
    decoder_inputs_emb = decoder_embedding_layer(decoder_inputs)
    y = decoder_inputs_emb
    decoder_lstm_layers = []
    for i in range(len(latent_dims)):
        layer = LSTM(latent_dims[i], kernel_regularizer=regularizers.l2(l2_lambda), return_sequences=True, dropout=dropout_rate, return_state=True)
        decoder_lstm_layers.append(layer)
        decoder_lstm_layers.append(layers.BatchNormalization())
        y, _, _ = layer(y, initial_state=encoder_states[2*i:2*i+2])
    
    # First LSTM layer (returns sequences for stacking)
    #decoder = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
    #decoder_outputs1, _, _ = decoder(decoder_inputs_emb, initial_state=encoder_states)
    
    # Second LSTM layer (returns sequences and states)
    #decoder_lstm2 = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
    #decoder_outputs, _, _ = decoder_lstm2(decoder_outputs1)
    
    output_dense_layer = Dense(outputs_numwords, activation='softmax')
    outputs = output_dense_layer(y)

/home/sid/miniconda3/envs/tensorflow-2.19_new/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1750705223.861691  317413 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


decoder_outputs_onehot shape: (43, 20, 214)


In [12]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 100)   │     15,500 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 20, 100)   │        400 │ embedding[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 20, 512), │  1,255,424 │ batch_normalizat… │
│                     │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 512)   │      2,048 │ lstm[0][0]        │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 20, 256), │    787,456 │ batch_normalizat… │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 256)   │      1,024 │ lstm_1[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 20, 512)   │    109,568 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, 20, 128), │    197,120 │ batch_normalizat… │
│                     │ (None, 128),      │            │                   │
│                     │ (None, 128)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, 20, 512), │  2,099,200 │ embedding_1[0][0… │
│                     │ (None, 512),      │            │ lstm[0][1],       │
│                     │ (None, 512)]      │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 128)   │        512 │ lstm_2[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ [(None, 20, 256), │    787,456 │ lstm_5[0][0],     │
│                     │ (None, 256),      │            │ lstm_1[0][1],     │
│                     │ (None, 256)]      │            │ lstm_1[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, 20, 64),  │     49,408 │ batch_normalizat… │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ [(None, 20, 128), │    197,120 │ lstm_6[0][0],   

 Total params: 5,583,794 (21.30 MB)

 Trainable params: 5,581,674 (21.29 MB)

 Non-trainable params: 2,120 (8.28 KB)

None


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', 
                                        min_delta=0.001,
                                        patience=25,
                                        restore_best_weights=True, 
                                        verbose=1,
                                        mode='auto')
 
trn = model.fit([encoder_input_sequences, decoder_input_sequences],
               decoder_outputs_onehot, 
               batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.1
                #,callbacks=[early_stopping_callback]
               )


Epoch 1/100


I0000 00:00:1750705240.815503  317503 cuda_dnn.cc:529] Loaded cuDNN version 90300


2/2 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.0028 - loss: 40.8966 - val_accuracy: 0.3900 - val_loss: 38.1183
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - accuracy: 0.0142 - loss: 37.9902 - val_accuracy: 0.3900 - val_loss: 35.4222
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.1631 - loss: 35.2872 - val_accuracy: 0.3900 - val_loss: 32.8914
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.3138 - loss: 32.7794 - val_accuracy: 0.3900 - val_loss: 30.5172
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.3757 - loss: 30.4506 - val_accuracy: 0.3900 - val_loss: 28.3159
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 0.4112 - loss: 28.2646 - val_accuracy: 0.3900 - val_loss: 26.2945
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step - accuracy: 0.4015 - loss: 26.2412 - val_accuracy: 0.3900 - val_loss: 24.4335
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - accuracy: 0.4168 - loss: 24.3658 - val_accuracy: 0.3900 - 

In [14]:

save_path = 'NUM_SENTENCES_' + str(NUM_SENTENCES) + "_EMBEDDING_SIZE_" + str(EMBEDDING_SIZE) + '_LSTM_NEURONS_' + str(LSTM_NEURONS) + '_model_mode_' + str(model_mode) 
folder_name = '/mnt/d/Work/wsl/ML-DS/saved_models/EnglishtoBengaliForMachineTranslation/model/' + save_path
print(folder_name)
# Save to a directory in the TensorFlow SavedModel format (recommended)
#model.save(folder_name+"/model.keras")

# Or save as a single HDF5 file (legacy Keras format)
#model.save(folder_name + '/model.h5')

/mnt/d/Work/wsl/ML-DS/saved_models/EnglishtoBengaliForMachineTranslation/model/NUM_SENTENCES_42_EMBEDDING_SIZE_100_LSTM_NEURONS_512_model_mode_2


In [15]:
encoder_model = Model(encoder_inputs, encoder_states)
print(encoder_model.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 20, 100)        │        15,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 100)        │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ [(None, 20, 512),      │     1,255,424 │
│                                 │ (None, 512), (None,    │               │
│                                 │ 512)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 512)        │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ [(None, 20, 256),      │       787,456 │
│                                 │ (None, 256), (None,    │               │
│                                 │ 256)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 20, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ [(None, 20, 128),      │       197,120 │
│                                 │ (None, 128), (None,    │               │
│                                 │ 128)]                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 20, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ [(None, 20, 64),       │        49,408 │
│                                 │ (None, 64), (None,     │               │
│                                 │ 64)]                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 20, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ [(None, 32), (None,    │        12,416 │
│                                 │ 32), (None, 32)]       │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,321,564 (8.86 MB)

 Trainable params: 2,319,444 (8.85 MB)

 Non-trainable params: 2,120 (8.28 KB)

None


In [19]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder inference model
# Define input layers for the initial states of each decoder LSTM layer
decoder_state_inputs = []
decoder_states = []
for dim in latent_dims:
    state_input_h = Input(shape=(dim,))
    state_input_c = Input(shape=(dim,))
    decoder_state_inputs.append(state_input_h)
    decoder_state_inputs.append(state_input_c)


decoder_input_word = Input(shape=(1,))
decoder_input_word_emb = decoder_embedding_layer(decoder_input_word)

y = decoder_input_word_emb
current_states = decoder_state_inputs # Start with the initial input states

for i in range(len(latent_dims)):
    # Get the current layer's initial states (h and c) from current_states
    layer_input_states = [current_states[2*i], current_states[2*i+1]]
    layer = decoder_lstm_layers[i * 2]
    y, state_h, state_c = layer(y, initial_state=layer_input_states)
    # Store the output states of the current layer
    decoder_states.append(state_h)
    decoder_states.append(state_c)


output_dense_layer = Dense(outputs_numwords, activation='softmax')
decoder_outputs = output_dense_layer(y)

# The decoder model takes the input word and all initial states, and outputs the predicted token probabilities and the final states
decoder_model = Model([decoder_input_word] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [ ]:
from numpy import argmax
from numpy import random
from numpy import array
import tensorflow as tf

index_to_word_input = {v:k for k,v in inputs_word2index.items()}
index_to_word_output = {v:k for k,v in outputs_word2index.items()}

def translate(input_seq):
    states = encoder_model.predict(input_seq, verbose=0)

    sos = outputs_word2index['<sos>']
    eos = outputs_word2index['<eos>']

    output_seq = zeros((1, 1))
    output_seq[0, 0] = sos

    output_sentence = []

    # Print shapes of decoder_model input layers for comparison
    # print("\nDecoder model input layer shapes:")
    # for i, input_layer in enumerate(decoder_model.inputs):
    #     print(f"  Input layer {i} shape: {input_layer.shape}")


    for _ in range(outputs_maxlen):
        # Pass the initial word and all states to the decoder model
        # Ensure all inputs have a batch size dimension and are TensorFlow tensors
        decoder_inputs = [tf.constant(output_seq, dtype=tf.float32)] + [tf.constant(state, dtype=tf.float32) for state in states]

        # print("Inputs to decoder_model.predict:", decoder_inputs)
        output_tokens_and_states = decoder_model.predict(decoder_inputs, verbose =0)

        output_tokens = output_tokens_and_states[0]
        states = output_tokens_and_states[1:] # Update states for the next time step
        # print("States from decoder_model.predict:", states)


        idx = argmax(output_tokens[0, 0, :])

        if idx == eos:
            break
        word = ''
        if idx > 0:
            word = index_to_word_output[idx]
            output_sentence.append(word)

        output_seq[0, 0] = idx # Update output_seq with the predicted word index

    return ' '.join(output_sentence)


from numpy import random

for _ in range(5):
    i = random.choice(len(inputs))
    input_seq = encoder_input_sequences[i:i+1]
    translation = translate(input_seq)

    print('Input:', inputs[i])
    print('Response:', translation)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 509ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 538ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Input: a man with gauges and glasses is wearing a blitz hat .
Response: কুকুর কুকুর কুকুর কুকুর ব্যক্তি কানওয়ালা কানওয়ালা কানওয়ালা ভিন্ন ভিন্ন ভিন্ন ভিন্ন ভিন্ন ভিন্ন মাথা মাথা মাথা মাথা মাথা মাথা
1/1

In [ ]:
my_inputs = ['not good']
my_inputs_seq = input_tokenizer.texts_to_sequences(my_inputs)
my_encoder_input_sequences = pad_sequences(my_inputs_seq, maxlen=inputs_maxlen)

translation = translate(my_encoder_input_sequences[0:1])
print(translation)